In [ ]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
import numpy as np
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [ ]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.shape[0]

In [ ]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #457467
ppp_geonull.head()

In [ ]:
ppp_geonull["BorrowerState"].value_counts()

In [ ]:
ppp_togeoIA = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "IA"]
ppp_togeoIA #60040

In [ ]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoIA_add = ppp_togeoIA[cols]
ppp_togeoIA_add

In [ ]:
add = ppp_togeoIA["full_add"].astype(str).to_list()

In [ ]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{2}[.]+\d+', result_text) #WEST 3-digit Longitude
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('19')] #IA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1

In [ ]:
print(len(add))
print(len(lat))
print(len(long))
print(len(geo_id))

<b>Add remaining values (2801) to result_df(5461)

In [ ]:
pd.set_option("display.max_rows", None)
lat_df = pd.DataFrame({"Lat":lat})
lat_df_drop = lat_df
lat_df_drop = lat_df_drop.drop(4558)


line = pd.DataFrame({"Lat": "42.05499",}, index=[3294.5])
df2 = lat_df_drop.append(line, ignore_index=False)
df2 = df2.sort_index().reset_index(drop=True)
lat_list = df2["Lat"].to_list()

In [ ]:
add_df = pd.DataFrame({"full_add":add, "Lat": lat_list})
add_df

In [ ]:
long_df = pd.DataFrame({"Long": long, "FIPS_z":geo_id})
long_df

In [ ]:
line = pd.DataFrame({"Long": "-93.62167", "FIPS_z":"191690003002005"}, index=[3294.5])
df3 = long_df.append(line, ignore_index=False)
df3 = df3.sort_index().reset_index(drop=True)
df3

In [ ]:
df4 = pd.concat([add_df, df3], axis=1)
df4

In [ ]:
result_df = df4

In [ ]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

In [ ]:
#mapping coordinates to NaN values in main 
m = dict(zip(result_df.full_add,result_df.Lat))
ppp_togeoIA.Lat = ppp_togeoIA.Lat.fillna(ppp_togeoIA.full_add.map(m))
n = dict(zip(result_df.full_add,result_df.Long))
ppp_togeoIA.Long = ppp_togeoIA.Long.fillna(ppp_togeoIA.full_add.map(n))
o = dict(zip(result_df.full_add,result_df.FIPS_z))
ppp_togeoIA.FIPS_z = ppp_togeoIA.FIPS_z.fillna(ppp_togeoIA.full_add.map(o))

In [ ]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoIA_parsed = ppp_togeoIA[cols]
ppp_togeoIA_parsed.head()

In [ ]:
ppp_geo = ppp_geo.merge(ppp_togeoIA_parsed, on="LoanNumber", how="left")
ppp_geo.head()

In [ ]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

In [ ]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo.csv")

In [ ]:
ppp_geo_ia = ppp_geo.loc[ppp_geo["BorrowerState"] =="IA"]
ppp_geo_ia.shape[0]

In [ ]:
null_check = ppp_geo_ia.loc[ppp_geo_ia["Lat"].isnull()]
null_check.shape[0]